In [1]:
import os


In [2]:
%pwd

'd:\\365 projects\\End-to-end-Machine-Learning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\365 projects\\End-to-end-Machine-Learning'

In [36]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [43]:
%pwd

'd:\\365 projects\\End-to-end-Machine-Learning'

In [44]:
import os
print(os.getcwd())


d:\365 projects\End-to-end-Machine-Learning


In [48]:
%cd "src\mlproject"

d:\365 projects\End-to-end-Machine-Learning\src\mlproject


In [51]:
%ls


 Volume in drive D has no label.
 Volume Serial Number is 8878-F613

 Directory of d:\365 projects\End-to-end-Machine-Learning\src\mlproject

05/17/2025  11:19 AM    <DIR>          .
05/17/2025  11:19 AM    <DIR>          ..
05/17/2025  11:27 AM               466 __init__.py
05/17/2025  02:32 PM    <DIR>          __pycache__
05/17/2025  12:01 AM    <DIR>          components
05/17/2025  12:01 AM    <DIR>          config
05/17/2025  02:32 PM    <DIR>          constants
05/17/2025  12:01 AM    <DIR>          entity
05/17/2025  12:01 AM    <DIR>          pipeline
05/17/2025  02:32 PM    <DIR>          utils
               1 File(s)            466 bytes
               9 Dir(s)  208,690,335,744 bytes free


In [52]:
%cd "constants/"

d:\365 projects\End-to-end-Machine-Learning\src\mlproject\constants


In [58]:
os.chdir("../")
%pwd

'd:\\365 projects\\End-to-end-Machine-Learning'

In [62]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [63]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [64]:
import os 
import urllib.request as request
import zipfile
from src.mlProject import logger 
from src.mlProject.utils.common import get_size

In [65]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [80]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-17 15:11:21,784: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-17 15:11:21,881: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-17 15:11:21,884: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-17 15:11:21,895: INFO: common: created directory at: artifacts]
[2025-05-17 15:11:21,897: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-17 15:11:23,270: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9319:3A22C4:D8DF77:F10343:68287D2D
Accept-Ranges: bytes
Date: Sat, 17